In [6]:
# Import Dependencies
import pandas as pd
import os
from sqlalchemy import create_engine
import time
import datetime

In [7]:
# Bring in CSVs
weather = 'Resources/Weather_Summary.csv'
wwii_data = 'Resources/THOR_WWII_DATA_CLEAN.csv'
wwii_aircraft_data = 'Resources/THOR_WWII_AIRCRAFT_GLOSS.csv'
wwii_weapons_data = 'Resources/THOR_WWII_WEAPON_GLOSS.csv'

In [12]:
# Read in Weather CSV 
weather_df = pd.read_csv(weather, encoding = "ISO-8859-1", low_memory=False)
weather_df.head()

,STA,Date,Precip,WindGustSpd,MaxTemp,MinTemp,MeanTemp,Snowfall,PoorWeather,YR,...,FB,FTI,ITH,PGT,TSHDSBRSGF,SD3,RHX,RHN,RVG,WTE
0,10001,1942-7-1,1.016,NaN,25.555556,22.222222,23.888889,0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10001,1942-7-2,0,NaN,28.888889,21.666667,25.555556,0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10001,1942-7-3,2.54,NaN,26.111111,22.222222,24.444444,0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10001,1942-7-4,2.54,NaN,26.666667,22.222222,24.444444,0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10001,1942-7-5,0,NaN,26.666667,21.666667,24.444444,0,NaN,42,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Read in Thor Weather CSV
thor_weather = pd.read_csv(wwii_data, encoding = "ISO-8859-1", low_memory=False)
thor_weather.head()

,WWII_ID,MASTER_INDEX_NUMBER,MSNDATE,THEATER,NAF,COUNTRY_FLYING_MISSION,TGT_COUNTRY_CODE,TGT_COUNTRY,TGT_LOCATION,TGT_TYPE,...,CALLSIGN,ROUNDS_AMMO,SPARES_RETURN_AC,WX_FAIL_AC,MECH_FAIL_AC,MISC_FAIL_AC,TARGET_COMMENT,MISSION_COMMENTS,SOURCE,DATABASE_EDIT_COMMENTS
0,1,NaN,8/15/1943,MTO,12 AF,USA,13.0,ITALY,SPADAFORA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4285,20028.0,2/20/1945,PTO,5 AF,USA,NaN,PHILIPPINE ISLANDS,PUERTA PRINCESA,UNIDENTIFIED TARGET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,NaN,8/15/1943,MTO,12 AF,USA,13.0,ITALY,COSENZA,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,NaN,8/15/1943,MTO,12 AF,USA,13.0,ITALY,GIOJA TAURO,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
4,8167,14639.0,2/23/1945,PTO,5 AF,USA,NaN,PHILIPPINE ISLANDS,BALETE PASS,WOODED AREA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Print Thor Weather columns
thor_weather.columns

Index(['WWII_ID', 'MASTER_INDEX_NUMBER', 'MSNDATE', 'THEATER', 'NAF',
       'COUNTRY_FLYING_MISSION', 'TGT_COUNTRY_CODE', 'TGT_COUNTRY',
       'TGT_LOCATION', 'TGT_TYPE', 'TGT_ID', 'TGT_INDUSTRY_CODE',
       'TGT_INDUSTRY', 'SOURCE_LATITUDE', 'SOURCE_LONGITUDE', 'LATITUDE',
       'LONGITUDE', 'UNIT_ID', 'MDS', 'AIRCRAFT_NAME', 'MSN_TYPE',
       'TGT_PRIORITY', 'TGT_PRIORITY_EXPLANATION', 'AC_ATTACKING', 'ALTITUDE',
       'ALTITUDE_FEET', 'NUMBER_OF_HE', 'TYPE_OF_HE', 'LBS_HE', 'TONS_OF_HE',
       'NUMBER_OF_IC', 'TYPE_OF_IC', 'LBS_IC', 'TONS_OF_IC', 'NUMBER_OF_FRAG',
       'TYPE_OF_FRAG', 'LBS_FRAG', 'TONS_OF_FRAG', 'TOTAL_LBS', 'TOTAL_TONS',
       'TAKEOFF_BASE', 'TAKEOFF_COUNTRY', 'TAKEOFF_LATITUDE',
       'TAKEOFF_LONGITUDE', 'AC_LOST', 'AC_DAMAGED', 'AC_AIRBORNE',
       'AC_DROPPING', 'TIME_OVER_TARGET', 'SIGHTING_METHOD_CODE',
       'SIGHTING_EXPLANATION', 'BDA', 'CALLSIGN', 'ROUNDS_AMMO',
       'SPARES_RETURN_AC', 'WX_FAIL_AC', 'MECH_FAIL_AC', 'MISC_FAIL_AC',
       

In [54]:
# Organize Thor weather data to list location as well as aircraft failures
thor_weather_fails = thor_weather[['LATITUDE','LONGITUDE','MSNDATE', 'WX_FAIL_AC', 'MECH_FAIL_AC', 'MISC_FAIL_AC']].copy()
thor_weather_fails

,LATITUDE,LONGITUDE,MSNDATE,WX_FAIL_AC,MECH_FAIL_AC,MISC_FAIL_AC
0,38.220000,15.370000,8/15/1943,NaN,NaN,NaN
1,9.750000,118.750000,2/20/1945,NaN,NaN,NaN
2,39.270000,16.250000,8/15/1943,NaN,NaN,NaN
3,38.430000,15.900000,8/15/1943,NaN,NaN,1.0
4,16.083333,120.916667,2/23/1945,NaN,NaN,NaN
...,...,...,...,...,...,...
178276,-8.472085,148.234293,12/14/1942,NaN,NaN,NaN
178277,4.883333,38.083333,9/18/1940,NaN,NaN,NaN
178278,-8.471111,126.458333,11/17/1942,NaN,NaN,NaN
178279,-2.583333,150.833333,2/16/1945,NaN,NaN,NaN


In [69]:
# Drop rows that don't have any failed aircraft
# Print clean Thor weather data
thor_weather_clean = thor_weather_fails.dropna(thresh = 1, subset = ['WX_FAIL_AC', 'MECH_FAIL_AC', 'MISC_FAIL_AC'])
thor_weather_clean

,LATITUDE,LONGITUDE,MSNDATE,WX_FAIL_AC,MECH_FAIL_AC,MISC_FAIL_AC
3,38.43,15.90,8/15/1943,NaN,NaN,1.0
1237,39.27,16.25,8/19/1943,1.0,NaN,NaN
1838,40.67,14.73,8/22/1943,NaN,NaN,10.0
2139,39.80,16.20,8/24/1943,12.0,NaN,NaN
3810,40.07,15.63,8/31/1943,NaN,1.0,NaN
...,...,...,...,...,...,...
176197,38.02,14.60,8/7/1943,4.0,NaN,NaN
176490,38.02,15.12,8/8/1943,2.0,NaN,NaN
177685,38.22,15.37,8/13/1943,NaN,NaN,2.0
178019,38.43,15.90,8/14/1943,NaN,NaN,3.0


In [70]:
# Organize Weather data to list weather components
weather_data_clean = weather_df[['Date', 'MaxTemp', 'MinTemp','MAX', 'MIN', 'Precip', 'WindGustSpd', 'Snowfall', 'PoorWeather', 'PRCP']].copy()

In [71]:
# Print clean weather data
weather_data_clean

,Date,MaxTemp,MinTemp,MAX,MIN,Precip,WindGustSpd,Snowfall,PoorWeather,PRCP
0,1942-7-1,25.555556,22.222222,78.0,72.0,1.016,NaN,0,NaN,0.04
1,1942-7-2,28.888889,21.666667,84.0,71.0,0,NaN,0,NaN,0
2,1942-7-3,26.111111,22.222222,79.0,72.0,2.54,NaN,0,NaN,0.1
3,1942-7-4,26.666667,22.222222,80.0,72.0,2.54,NaN,0,NaN,0.1
4,1942-7-5,26.666667,21.666667,80.0,71.0,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...
119035,1945-12-27,28.333333,18.333333,83.0,65.0,0,NaN,0,NaN,0
119036,1945-12-28,29.444444,18.333333,85.0,65.0,9.906,NaN,0,1,0.39
119037,1945-12-29,28.333333,18.333333,83.0,65.0,0,NaN,0,1,0
119038,1945-12-30,28.333333,18.333333,83.0,65.0,0,NaN,0,NaN,0


In [72]:
# Convert dates to proper formatting
weather_dates = weather_data_clean['Date'] = pd.to_datetime(weather_data_clean['Date'])

In [73]:
# Print updated clean Thor weather data
thor_weather_clean['MSNDATE'] = pd.to_datetime(thor_weather_clean['MSNDATE'])
thor_weather_clean

/Users/ibethernandez/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,LATITUDE,LONGITUDE,MSNDATE,WX_FAIL_AC,MECH_FAIL_AC,MISC_FAIL_AC
3,38.43,15.90,1943-08-15,NaN,NaN,1.0
1237,39.27,16.25,1943-08-19,1.0,NaN,NaN
1838,40.67,14.73,1943-08-22,NaN,NaN,10.0
2139,39.80,16.20,1943-08-24,12.0,NaN,NaN
3810,40.07,15.63,1943-08-31,NaN,1.0,NaN
...,...,...,...,...,...,...
176197,38.02,14.60,1943-08-07,4.0,NaN,NaN
176490,38.02,15.12,1943-08-08,2.0,NaN,NaN
177685,38.22,15.37,1943-08-13,NaN,NaN,2.0
178019,38.43,15.90,1943-08-14,NaN,NaN,3.0


In [ ]:
# Rename columns to make dataframes cleaner
weather_df2 = weather_df2.rename(columns= {"Date":'dates','MAX':'maxtempf','MIN':'mintempf', 'MaxTemp': 'maxtempc', 'MinTemp': 'mintempc'})

In [ ]:
weather_df2